# Info-clustering and Game-Clustering
This is a demonstration of the info-clustering and game-clustering algorithm reduced to weighted graphs. You may use the binder service to run this notebook.



## Preliminaries

Run the following cell to import the required library.

In [ ]:
import game_clustering
import infoclustering
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

The following are functions that help present the clustering solutions. You can modify them to show the results according to your preferences.

In [ ]:
def get_modularity_from_cluster(G, cluster):
    get_id_from_vertex = {}
    for i, v in enumerate(G.nodes):
        get_id_from_vertex[v] = i
    # map node id to index id (the node may not start from zero, continuous from 0 to |N|, e.t.c)
    cluster = [[get_id_from_vertex[v] for v in c] for c in cluster]
    # cluster is a list of list
    B = nx.modularity_matrix(G)
    modularity = 0
    for community in cluster:
        c = list(community)
        modularity += np.sum(B[c][:, c])
    modularity = modularity / (2 * G.number_of_edges())
    return modularity

def print_game_solution(game_solution_dict):
    # solution_dict is a dictionary, keys are the tensile strength, values are the corresponding cluster
    for alpha, cluster in sorted(game_solution_dict.items()):
        print('alpha: ', alpha)
        print('cluster: ', [set(community) for community in cluster])
        print()
        
def print_info_solution(info_solution_dict):
    for i, ((gamma_1, gamma_2), cluster) in enumerate(sorted(info_solution_dict.items())):
        if i == 0:
            print('for MMI in (', gamma_1, ', ', gamma_2, ']')
        else:
            print('for MMI in (', gamma_1, ', ', gamma_2, ']')
        print([set(community) for community in cluster])
        print()

## Examples
We can now run the clustering algorithms on some simple examples and compare them with other algorithms.

### A simple graph with a triangle and a line

The following defines a digraph and its undirected counterpart.

In [ ]:
D = nx.DiGraph()
D.add_edges_from([(0, 1, {'weight': 1}), 
                   (1, 2, {'weight': 1}), 
                   (2, 0, {'weight': 1}), 
                   (3, 4, {'weight': 1})])
G = D.to_undirected()

#### Info-clustering on the undirected graph

To run the info-clustering algorithm:

In [ ]:
ic = infoclustering.InfoClustering(G=G)
ic.fit()

To obtain the documentation:

In [ ]:
help(ic)

The clusters are stored in the following object:

In [ ]:
ic.solutions

Note that, other than the trivial clusters (singleton sets or the trival sets), info-clustering returns the desired clusters {3,4} and {0,1,2}. Let's pretty-print the solution using the helper function defined before.

In [ ]:
print_info_solution(ic.solutions)

#### Game-clustering on the digraph

To run the game-clustering:

In [ ]:
gc = game_clustering.GameClustering(G=D, beta=1, weight='weight')
gc.fit()

To obtain the documentation:

In [ ]:
help(gc)

To get the clustering solution:

In [ ]:
gc.solutions

We can again pretty print the solution using the helper function defined earlier:

In [ ]:
print_game_solution(gc.solutions)

### Game-clustering vs Info-clustering
The following example shows that game-clustering can depend on the direction of the links, unlike info-clustering. 

In [ ]:
D = nx.DiGraph()
D.add_edges_from([(1, 0, {'weight': 2}), 
                   (1, 2, {'weight': 1}), 
                   (2, 1, {'weight': 1})])
G = nx.Graph()
G.add_edges_from([(1, 0, {'weight': 2}), 
                   (1, 2, {'weight': 2})])

Note that we need a separate definition for the undirected graph because `to_undirected` only keeps one of the two directed edges (1,2) and (2,1), instead of merging them into an undirected edge.

In [ ]:
ic = infoclustering.InfoClustering(G=G)
ic.fit()
print_info_solution(ic.solutions)

Note that info-clustering does not return any non-trivial community.

In [ ]:
gc = game_clustering.GameClustering(G=D, beta=0, weight='weight')
gc.fit()
print_game_solution(gc.solutions)

Game-clustering, on the contrary, returns the community {1,2}, which has more internal links than external incoming links.

### Ring Of Cliques

In [ ]:
G = nx.ring_of_cliques(num_cliques=30, clique_size=5)

In [ ]:
gc = game_clustering.GameClustering(G=G, beta=1)
gc.fit()

In [ ]:
print_game_solution(gc.solutions)

In [ ]:
# for info-clustering
ic = infoclustering.InfoClustering(G=G)
ic.fit()
print_info_solution(IC.solutions)

In [ ]:
# greedy modularity
modularity_cluster = nx.algorithms.community.modularity_max.greedy_modularity_communities(G)
modularity_cluster

# modularity approach will group 2 consecutive cliques together

In [ ]:
every_clique = get_modularity_from_cluster(G, gc.solutions[3.0937931])
every_two_clique = get_modularity_from_cluster(G, modularity_cluster)
print('Modularity of grouping every clique: ', every_clique)
print('Modularity of grouping every two consecutive clique: ', every_two_clique)
print('Modularity of grouping every two clique is larger than the modularity of grouping every clique: ', every_two_clique>every_clique)

# Modularity is higher if we group two consecutive cluster together, but seperating every clique is desired.

### Web community

In [ ]:
G = nx.grid_graph([7, 7])
T = G.copy()
for i, j in G.edges:
    if (i[1]==0 and j[1]==0 and T.has_edge(i, j)) or \
        (i[1]==1 and j[1]==1 and T.has_edge(i, j)) or \
        (i[1]==5 and j[1]==5 and T.has_edge(i, j)) or \
        (i[1]==6 and j[1]==6 and T.has_edge(i, j)) or \
        (i[0]==0 and j[0]==0 and T.has_edge(i, j)) or \
        (i[0]==1 and j[0]==1 and T.has_edge(i, j)) or \
        (i[0]==5 and j[0]==5 and T.has_edge(i, j)) or \
        (i[0]==6 and j[0]==6 and T.has_edge(i, j)):
        # remove first column
        T.remove_edge(i, j)
T.remove_node((0,0))
T.remove_node((0,1))
T.remove_node((1,0))
T.remove_node((1,1))

T.remove_node((0,5))
T.remove_node((0,6))
T.remove_node((1,5))
T.remove_node((1,6))
# last column top and bottom
T.remove_node((5,0))
T.remove_node((6,0))
T.remove_node((5,1))
T.remove_node((6,1))

T.remove_node((5,5))
T.remove_node((6,6))
T.remove_node((5,6))
T.remove_node((6,5))
T.number_of_edges()

# add weight
nx.set_edge_attributes(T, 1, 'weight')
web_center_row_column = [2,3,4]
for i, j in T.edges:
    if (i[0] in web_center_row_column) \
    and (i[1] in web_center_row_column) \
    and (j[0] in web_center_row_column) \
    and (j[1] in web_center_row_column):
        T[i][j]['weight'] = 1
# relabel the nodes
k = 0
pos = {}
for node in sorted([i for i in T], key=lambda element: (element[0], -element[1]))[::-1]:
    pos[k] = ([node[1], node[0]])
    T = nx.relabel_nodes(T, {node: k})
    k += 1
G=T.copy()
nx.draw_networkx_labels(G,pos=pos,labels={i:i for i in (G)},font_size=13)
nx.draw(G,pos=pos, node_color='w')
plt.show()
plt.clf()

In [ ]:
gc = game_clustering.GameClustering(G=G, beta=0.8)
gc.fit()

In [ ]:
print_game_solution(gc.solutions)

In [ ]:
# for info-clustering
IC = infoclustering.InfoClustering(G=G)
IC.fit()
print_info_solution(IC.solutions)

# Web Spider Graph with 2 web center

In [ ]:
G = nx.grid_graph([10, 5])
T = G.copy()
for i, j in G.edges:
    if (i[1]==0 and j[1]==0) or\
    (i[1]==9 and j[1]==9) or\
    (i[1]==4 and j[1]==4) or\
    (i[1]==5 and j[1]==5) or\
    (i[0]==0 and j[0]==0) or\
    (i[0]==4 and j[0]==4):
        # remove first column
        # remove last column
        # remove middle column
        # remove first row
        # remove last row
        T.remove_edge(i, j)
# first column top and bottom two node
T.remove_node((0,0))
T.remove_node((4,0))
# middle column top and bottom two node
T.remove_node((0,4))
T.remove_node((4,4))
T.remove_node((0,5))
T.remove_node((4,5))
# last column top and bottom two node
T.remove_node((0,9))
T.remove_node((4,9))
T.number_of_edges()

k = 0
pos = {}
for node in sorted([i for i in T], key=lambda element: (element[0], -element[1]))[::-1]:
    pos[k] = ([node[1], node[0]])
    T = nx.relabel_nodes(T, {node: k})
    k += 1
G=T.copy()
labels = {}
nx.draw_networkx_labels(G,pos=pos,labels={i:i for i in (G)},font_size=13)
nx.draw(G,pos=pos, node_color='w')
plt.show()

In [ ]:
gc = game_clustering.GameClustering(G=G, beta=0.8)
gc.fit()

In [ ]:
print_game_solution(gc.solutions)

In [ ]:
# for info-clustering
IC = infoclustering.InfoClustering(G=G)
IC.fit()
print_info_solution(IC.solutions)